In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

## 定义输入数据，并预处理数据

In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
tr_x,tr_y,te_x,te_y = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

In [4]:
tr_x = tr_x.reshape(-1,28,28,1) # 28*28*1 imput img
te_x = te_x.reshape(-1,28,28,1)

In [5]:
X = tf.placeholder("float",[None,28,28,1])
Y = tf.placeholder("float",[None,10])

## 初始化权重与定义网络结构

In [6]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))

In [7]:
w = init_weights([3,3,1,32])
w2 = init_weights([3,3,32,64])
w3 = init_weights([3,3,64,128])
w4 = init_weights([128*4*4,625])
w_o = init_weights([625,10])

In [8]:
def model(X,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X,w,strides=[1,1,1,1],padding='SAME'))
    l1 = tf.nn.max_pool(l1a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l1 = tf.nn.dropout(l1,p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1,w2,strides=[1,1,1,1],padding='SAME'))
    l2 = tf.nn.max_pool(l2a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l2 = tf.nn.dropout(l2,p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2,w3,strides=[1,1,1,1],padding='SAME'))
    l3 = tf.nn.max_pool(l3a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l3 = tf.reshape(l3,[-1,w4.get_shape().as_list()[0]])
    l3 = tf.nn.dropout(l3,p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3,w4))
    l4 = tf.nn.dropout(l4,p_keep_hidden)
    
    pyx = tf.matmul(l4,w_o)
    
    return pyx

In [9]:
p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hidden)

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.argmax(py_x,1)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## 训练模型与评估模型

In [11]:
batch_size = 128
test_size = 256

In [12]:
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        
        for i in range(30):
            training_batch = zip(range(0,len(tr_x),batch_size),range(batch_size,len(tr_x)+1,batch_size))
            for start,end in training_batch:
                sess.run(train_op,feed_dict={X:tr_x[start:end],Y:tr_y[start:end],p_keep_conv:0.8,p_keep_hidden:0.5})
            
            test_indices = np.arange(len(te_x))
            np.random.shuffle(test_indices)
            test_indices = test_indices[0:test_size]
            
            print(i,np.mean(np.argmax(te_y[test_indices],axis=1) == 
                            sess.run(predict_op,feed_dict={X:te_x[test_indices],p_keep_conv:1.0,p_keep_hidden:1.0})))
            

0 0.94921875
1 0.99609375
2 0.98046875
3 0.99609375
4 0.9921875
5 0.98046875
6 0.99609375
7 0.9921875
8 0.9921875
9 0.98828125
10 0.98828125
11 0.984375
12 0.98046875
13 0.98828125
14 0.98828125
15 0.99609375
16 0.9921875
17 0.98828125
18 0.98828125
19 0.98828125
20 0.98046875
21 0.99609375
22 0.99609375
23 1.0
24 0.99609375
25 0.99609375
26 0.99609375
27 0.99609375
28 0.99609375
29 0.99609375
